In [1]:
from wordfreq import zipf_frequency
import tensorflow as tf
import math

msg = tf.constant('Hello, TensorFlow!... finally')
tf.print(msg)

%matplotlib inline
import re
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from functools import reduce
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#from sklearn.model_selection import train_test_split

Hello, TensorFlow!... finally


Using TensorFlow backend.


In [5]:
n = ['id', 'url_legal', 'license', 'excerpt','target','standard_error']
data_train = pd.read_csv('train.csv', sep=',',
                         error_bad_lines=False, names=n,
                         quotechar='"',skipinitialspace=True,
                         usecols=['excerpt','standard_error','target'],skiprows=1)


In [10]:
# zipf freq is basicaly log_10(freq * 10^9) = 9 + log_10(freq)
# i.e zipf = 6 is once per 1000
# 9 + log10(1/1000) = 6

zipf_frequency('ubiquitous', 'en')
print(data_train['standard_error'].max())
print(data_train['standard_error'].min())
print(data_train['standard_error'].mean())

0.649671297
0.0
0.49143509098976706


In [82]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[\n,.-]+', ' ', text)
    text = re.sub(r'\b[a-z]{,3}\b', ' ', text)
    text = text = re.sub(r'[\s]+', ' ', text)
    return text.strip()

def get_freq_list(text):
    return list( map(lambda x: zipf_frequency(x, 'en') , text.split(" ")))
     
def get_excerpt_cmplx(freq_list):
    return reduce(lambda x, y:  x + ( 1 if y <= 4 else 0), freq_list,0)

data_train['excerpt'][0]
text1 = data_train['excerpt'][0]
text1 = preprocess_text(text1)
print(text1)
freq_list = get_freq_list(text1)
print(freq_list)

res = get_excerpt_cmplx(freq_list)
print(res)

when young people returned ballroom presented decidedly changed appearance instead interior scene winter landscape floor covered with snow white canvas laid smoothly rumpled over bumps hillocks like real snow field numerous palms evergreens that decorated room were powdered with flour strewn with tufts cotton like snow also diamond dust been lightly sprinkled them glittering crystal icicles hung from branches each room wall hung beautiful bear skin these rugs were prizes girls boys this game girls were gathered room boys other called north pole other south pole each player given small flag which they were plant reaching pole this would have been easy matter each traveller obliged wear snowshoes
[6.37, 5.43, 6.25, 4.82, 3.52, 4.75, 3.37, 5.07, 4.69, 5.24, 4.55, 4.97, 4.89, 4.36, 4.94, 4.83, 6.85, 4.66, 5.51, 3.96, 4.56, 3.68, 2.24, 6.08, 3.6, 1.96, 6.41, 5.6, 4.66, 5.24, 4.65, 3.55, 2.33, 7.01, 3.94, 5.39, 6.34, 3.31, 6.85, 4.0, 3.08, 6.85, 3.1, 4.35, 6.41, 4.66, 6.19, 4.42, 4.41, 6.27,

In [40]:
text2 = 'When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThese rugs were for prizes, one for the girls and one for the boys. And this was the game.\nThe girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.\nThis would have been an easy matter, but each traveller was obliged to wear snowshoes.'
text3 = 'When the not young \npeople returned to the ballroom, it presented a'

cols = ['dot','com','wcnt','text','cmpl']
df = pd.DataFrame(columns=cols)  

for i in range(len(data_train)):
    text = data_train['excerpt'][i]
    processed_text = preprocess_text(text)
    df = df.append({
         "dot": text.count('.'),
         "com":  text.count(','),
         "text": processed_text,
         "wcnt": len(text.split()),
         "cmpl": get_excerpt_cmplx(get_freq_list(processed_text))
          }, ignore_index=True)



In [41]:
df.loc[0]

dot                                                    11
com                                                    14
wcnt                                                  179
text    when young people returned ballroom presented ...
cmpl                                                   22
Name: 0, dtype: object

In [42]:
df

,dot,com,wcnt,text,cmpl
0,11,14,179,when young people returned ballroom presented ...,22
1,10,24,169,through dinner time fayre somewhat silent eyes...,40
2,11,17,166,roger predicted snow departed quickly came day...,34
3,5,23,164,outside before palace great garden walled roun...,32
4,5,13,147,once upon time there were three bears lived to...,11
...,...,...,...,...,...
2829,10,12,145,when think dinosaurs where they lived what pic...,27
2830,13,5,164,what solid ? solids usually hard because their...,20
2831,16,2,190,second state matter will discuss liquid solids...,15
2832,12,8,150,solids shapes that actually touch they have th...,24


In [62]:
# n = ['id', 'url_legal', 'license', 'excerpt','target','standard_error']
# id,url_legal,license,excerpt,target,standard_error

# data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
# data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# sample_size = min(data_positive.shape[0], data_negative.shape[0])
# raw_data = np.concatenate((data_positive['text'].values[:sample_size],
#                            data_negative['text'].values[:sample_size]), axis=0)
# labels = [1] * sample_size + [0] * sample_size
# data_int = [preprocess_text(t) for t in raw_data]


# length = [len(i) for i in data_int]
# print("Average Review length:", np.mean(length))
# print("Standard Deviation:", round(np.std(length)))
SENTENCE_LENGTH = 5
foo = [1,2]
foo[0] = [ i for i in range(10)]
foo[1] = [ i for i in range(10,20)]

print(foo)    
print(pad_sequences(foo, maxlen=SENTENCE_LENGTH)) 

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]
[[ 5  6  7  8  9]
 [15 16 17 18 19]]


In [7]:
SENTENCE_LENGTH = 50
NUM = 10000
NUM_TEST = 50000

def get_sequences(data_p):
    sequences = tokenizer.texts_to_sequences(data_p)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

#we will use only NUM top used words as comp nerezinovi
def get_my_sequences(comment_p):
    result = [index.get(str,1) for str in comment_p.split(' ') if index.get(str,1) < NUM]
    return result

def vectorize(sequences, dimension = NUM):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results


tokenizer = Tokenizer(num_words=NUM,oov_token="#")
tokenizer.fit_on_texts(data_int[NUM:])

index = tokenizer.word_index

#Change words to indexes from vocabulary that we sdelali
data = [get_my_sequences(comment) for comment in data_int]

#Change to 0-1 index
data = vectorize(data)
targets = np.array(labels).astype("float32")
        
#this is keras train/test it is with shuffle
train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size=0.2, random_state=2)

#test_x = data[:NUM_TEST]
#test_y = targets[:NUM_TEST]

#train_x = data[NUM_TEST:]
#train_y = targets[NUM_TEST:]

In [8]:
i=3
strTest = "USER углу сидит погибает голода еще порции взяли хотя уже жрать не хотим"
ttt1 = [index.get(str,1) for str in strTest.split(' ')]
print(index.get('школота'))
print(index.get('поверь'))
print(len(index))

print('data:_____')
print(strTest)
print(ttt1)
print(get_my_sequences(strTest))
print(data[i][0:60])


13866
2892
170145
data:_____
USER углу сидит погибает голода еще порции взяли хотя уже жрать не хотим
[1, 5005, 759, 163604, 6285, 10, 16649, 1653, 97, 11, 889, 3, 2216]
[1, 5005, 759, 6285, 10, 1653, 97, 11, 889, 3, 2216]
[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


-3 is some magic.# replace when word not found

In [9]:
model = models.Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                500050    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 505,201
Trainable params: 505,201
Non-trainable params: 0
________________________________________________

In [10]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [11]:
results = model.fit(
 train_x, train_y,
 epochs= 2,
 batch_size = 500,
 validation_data = (test_x, test_y)
)


Train on 179076 samples, validate on 44770 samples
Epoch 1/2
179076/179076 [==============================] - 60s 336us/step - loss: 0.5735 - accuracy: 0.6967 - val_loss: 0.5330 - val_accuracy: 0.7306
Epoch 2/2
179076/179076 [==============================] - 64s 360us/step - loss: 0.4992 - accuracy: 0.7544 - val_loss: 0.5285 - val_accuracy: 0.7341


In [12]:
print(np.mean(results.history["val_accuracy"]))


0.7323542535305023


In [13]:
testPositive = "прекрасный наполненный смыслом и радостью фильм понравился очень"
testNegative = "этот фильм полный шлак посмотрел самый худший фильм видел оценка минус один нуля"

print(testPositive)
print(testNegative)

testPositiveArr = [1]
testNegativeArr = [1]

print("\n"+"Get index "+"\n")

testPositiveArr[0] = get_my_sequences(testPositive)
testNegativeArr[0] = get_my_sequences(testNegative)

print(testPositiveArr)
print(testNegativeArr)

прекрасный наполненный смыслом и радостью фильм понравился очень
этот фильм полный шлак посмотрел самый худший фильм видел оценка минус один нуля

Get index 

[[1405, 1, 8683, 2385, 179, 936, 18]]
[[68, 179, 1084, 784, 182, 6773, 179, 493, 5039, 1132, 82]]


In [14]:
testPositiveArrINDX = vectorize(testPositiveArr)
testNegativeArrINDX = vectorize(testNegativeArr)

predictPositive = model.predict(testPositiveArrINDX)
predictNegative = model.predict(testNegativeArrINDX)

print('Positive: ' + str(predictPositive[0][0]))
print('Negative: ' + str(predictNegative[0][0]))

Positive: 0.8635264
Negative: 0.051291257


In [15]:
model.save('trainedModel.hdf5')
df = pd.DataFrame.from_dict(index, orient="index")
df.to_csv("commentDict.csv")